Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [ ]:
import retinoto_py as fovea
args = fovea.Params(do_fovea=True)
args

# testing each network on the validation dataset

In [ ]:
%ls cached_data/32*

In [ ]:
# %rm cached_data/32_fovea_model_*  # FORCING RECOMPUTE
# %rm cached_data/32_fovea_model_*.lock  # FORCING RECOMPUTE
model_name = args.model_name
# for dataset in fovea.all_datasets:
for dataset in ['bbox']:# <HACK to go fat-ster ot the point
    name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

## Model comparison

In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

In [ ]:
%ls cached_data/20*

In [ ]:
model_name, dataset

In [ ]:
# results_20 = fovea.pd.read_json(args.data_cache / '20_model_name={model_name}_dataset={dataset}_retrained.json')
# results_20

In [ ]:
# for dataset in fovea.all_datasets:
#     name = f'20_model_name={model_name}_dataset={dataset}_retrained'
#     model_filename, json_filename = fovea.do_learning(args, dataset, name)
#     results = fovea.pd.read_json(json_filename)
#     print(results)

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in ['bbox']:#fovea.all_datasets:
    color = 'r'
    lw = 1

    # name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    # cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.json
    json_filename = args.data_cache / f'32_fovea_model_name={model_name}_dataset={dataset}_retrained.json'
    # model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    # df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train

In [ ]:
json_filename

In [ ]:
%pwd

In [ ]:
fovea.get_dataset??